# Keras Basic
### * Sequential Layer 의 주요 레이어 정리 
#### [1]  Dense Layer : Fully Connected Layer, layer의 입력과 출력 사이에 있는 모든 뉴런이 서로 연결 되는 Layer (Affine[일반식 : W*X + b = H(x)], matmul, shape 중요)
#### [2]  Flatten Layer : 다차원(4차원)을 2차원으로 축소하여 FC Layer에 전달한다
#### [3]  Conv2D Layer : 이미지 특징을 추출하는 Convolution Layer 
#### [4]  MaxPool2D Layer : 중요 데이터(값이 큰 데이터)를 subsampling 하는 Layer
#### [5]  Dropout Layer : 학습시 신경망의 과적합을 막기위해 일부 뉴런을 제거하는 Layer|

In [2]:
import tensorflow as tf
import numpy as np

In [2]:
# keras 를 이용한 XOR 네트워크

# train data set 
x_data = [[0,0],
          [0,1],
          [1,0],
          [1,1]]   # (4,2)

y_data = [[0],
          [1],
          [1],
          [0]]     # (4,1)

x_train = np.array(x_data,dtype=np.float32)
y_train = np.array(y_data,dtype=np.float32)

In [21]:
# Dense Layer 구현
model = tf.keras.Sequential([
    # (4,2) * (2,4) = (4,4) l1
    # (4,4) * (4,1) = (4,1) l2
    tf.keras.layers.Dense(units=4, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 4)                 12        
                                                                 
 dropout (Dropout)           (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 17
Trainable params: 17
Non-trainable params: 0
_________________________________________________________________


In [22]:
# 학습
model.fit(x_train,y_train,epochs=1000)

Epoch 1/1000
1/1 [==============================] - 0s 366ms/step - loss: 0.8531 - accuracy: 0.2500
Epoch 2/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.8049 - accuracy: 0.2500
Epoch 3/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.7325 - accuracy: 0.5000
Epoch 4/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.7093 - accuracy: 0.5000
Epoch 5/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.7446 - accuracy: 0.2500
Epoch 6/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.8248 - accuracy: 0.2500
Epoch 7/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.7063 - accuracy: 0.5000
Epoch 8/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6120 - accuracy: 0.7500
Epoch 9/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.8086 - accuracy: 0.2500
Epoch 10/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.7825 - accuracy: 0.2500
Epoch 11/1000
1/1

In [7]:
# 예측
preds = model.predict(x_train)
np.round(preds)

1/1 [==============================] - 0s 17ms/step


array([[0.],
       [1.],
       [0.],
       [0.]], dtype=float32)

In [8]:
model.evaluate(x_train,y_train)

1/1 [==============================] - 0s 127ms/step - loss: 0.3472 - accuracy: 0.7500


[0.34723547101020813, 0.75]

# [2] Flatten Layer

In [13]:
# 4차원 데이터를 2차원으로 축소하여 FC Layer에 전달
data = np.arange(1000*28*28*10).reshape(1000,28,28,10).astype(np.float32)
print(data.shape) # (None, 28, 28, 10)

layer = tf.keras.layers.Flatten()
outputs = layer(data)
print(outputs.shape)

(1000, 28, 28, 10)
(1000, 7840)


In [16]:
# 3차원 데이터를 2차원으로 축소하여 FC Layer에 전달
data = np.arange(1000*28*28).reshape(1000,28,28).astype(np.float32)
print(data.shape) # (None, 28, 28)

layer = tf.keras.layers.Flatten() # (None, 784)
outputs = layer(data)
print(outputs.shape)

(1000, 28, 28)
(1000, 784)


In [18]:
# 2차원 데이터를 2차원으로 축소 X
data = np.arange(1000*28).reshape(1000,28).astype(np.float32)
print(data.shape) # (None, 28)

layer = tf.keras.layers.Flatten() # (1000, 28)
outputs = layer(data)
print(outputs.shape)

(1000, 28)
(1000, 28)


### [3] Dropout Layer

In [20]:
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout
# The Dropout layer randomly sets input units to 0 with a frequency of rate 
# at each step during training time, which helps prevent overfitting. 
### Inputs not set to 0 are scaled up by 1/(1 - rate) such that the sum over all inputs is unchanged.

# Note that the Dropout layer only applies when training is set to True such that
# no values are dropped during inference. When using model.fit, training will be
# appropriately set

In [3]:
tf.random.set_seed(0)
data = np.arange(10).reshape(5,2).astype(np.float32)
print(data)

layer = tf.keras.layers.Dropout(0.3,input_shape=(2,))

outputs = layer(data,training=True)
# model.fit()호출 시는  학습 모드로 training=True가 되어 dropuout 적용
# model.evaluate() 호출 시는 예측 모드로 False가 되어 dropuout이 수행되지 않음
print(outputs) # 데이터의 30%는 0으로 나머지 데이터는  1/(1-0.3)으로 곱하여 scaled up된다
print(data*1/(1 - 0.3))

[[0. 1.]
 [2. 3.]
 [4. 5.]
 [6. 7.]
 [8. 9.]]
tf.Tensor(
[[ 0.         0.       ]
 [ 2.857143   4.285714 ]
 [ 5.714286   7.1428576]
 [ 8.571428  10.       ]
 [11.428572   0.       ]], shape=(5, 2), dtype=float32)
[[ 0.         1.4285715]
 [ 2.857143   4.285714 ]
 [ 5.714286   7.142857 ]
 [ 8.571428  10.       ]
 [11.428572  12.857143 ]]
